<a href="https://colab.research.google.com/github/Atellas23/notebooks/blob/main/MarkovDisseminationProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.random.seed(123456)

from tqdm import tqdm
!pip install pipe -q
from pipe import select, where
import altair as alt
import pandas as pd

In [ ]:
# size of the dataset and dimensions
N = 200
d = 2
# dataset
random_points1 = np.random.multivariate_normal(-2*np.ones(d), 4*np.eye(d), N)
random_points2 = np.random.multivariate_normal(10*np.ones(d), 4*np.eye(d), N)
random_points = np.concatenate((random_points1, random_points2))
data = pd.DataFrame({
    'x': random_points[:, 0],
    'y': random_points[:, 1]
})
# random_points2 = np.random.multivariate_normal(5*np.ones(2), 4*np.array([[1, -1], [-1, 1]]), N)

In [ ]:
alt.Chart(data).mark_circle().encode(
    x='x',
    y='y',
    color=alt.value('black')
)

alt.Chart(...)

In [ ]:
class MarkovDisseminationProcess:
    def __init__(self, data, gamma=-0.02, iterations=1000, p=0.999, distance_option='mean'):
        self.num_it = iterations
        self.data = data
        self.clusters = [set(range(len(data)))] # initialize with total cluster
        self.gamma = gamma
        self.p = p
        self.cluster = np.zeros(len(data))
        self.option = distance_option
    
    def dist(self, a, b):
        return np.sqrt(np.sum(np.square(a-b)))

    def dist_point_clust(self, point, clust):
        if self.option is 'mean':
            return np.mean([self.dist(point, self.data[b]) for b in self.clusters[clust]])
        elif self.option is 'random':
            # k = np.random.randint(len(self.clusters[clust]))
            k = np.random.choice(list(self.clusters[clust])) # select a random element from cluster
            return self.dist(point, self.data[k]) # calculate distance to the random element
        elif self.option is 'min':
            return np.min([self.dist(point, self.data[b]) for b in self.clusters[clust]])
        elif self.option is 'any':
            if len(self.clusters[clust]) == 0:
                return np.inf
            el = self.clusters[clust].copy().pop()
            return self.dist(point, el)

    def mean_dist_to_clust(self, point, clust):
        return np.mean([self.dist(point, self.data[b]) for b in self.clusters[clust]])
    
    def cluster_distances(self, point):
        return sorted([(clust, self.dist_point_clust(point, clust)) for clust in range(len(self.clusters))], key=lambda t: t[1])

    def current_cluster(self, point_idx):
        return int(self.cluster[point_idx])

    def step(self, iteration, debug=False):
        for i, point in enumerate(self.data):
            # decide whether to change cluster or to stay
            p = np.random.random()
            stay = p > 0.5*np.exp(self.gamma*iteration)
            if not stay:
                # decide whether to form a new cluster or to jump to another one
                p = np.random.random()
                new_clust = p > self.p
                if not new_clust:
                    # compute distance to all clusters
                    distances = self.cluster_distances(point)
                    # get the closest cluster
                    closest_cluster = distances[0][0]
                    current_cluster = self.current_cluster(i)
                    if closest_cluster != current_cluster:
                        # jump to the closest cluster
                        self.clusters[current_cluster].remove(i)
                        self.clusters[closest_cluster].update({i})
                        self.cluster[i] = closest_cluster
                    # If the closest cluster was the current cluster,
                    # jumping is the same as not doing anything.
                    # Hence, we do nothing, and we reduce the remove-
                    # update overhead.
                else:
                    # form a new cluster
                    current_cluster = self.current_cluster(i)
                    self.clusters[current_cluster].remove(i)
                    self.cluster[i] = len(self.clusters)
                    self.clusters.append({i})


    def run(self, debug=False):
        print('Starting a Markov Dissemination Process')
        for it in tqdm(range(self.num_it)):
            self.step(it, debug=debug)
            if debug and it % (self.num_it//5) == 0:
                print(f'iteration {it}/{self.num_it}')
                print(f'current number of clusters: {len(self.clusters)}')
        print(f'Ended Markov Dissemination Process with {len(self.clusters)} clusters')

    def generate_cluster_inverse_dict(self):
        res = {}
        for cluster_index, cluster in enumerate(self.clusters):
            for point_index in cluster:
                res[point_index] = cluster_index
        res = res.items()
        res = list(sorted(res, key=lambda t: t[0]) | select(lambda t: t[1]))
        return res
    
    def get_clusters(self):
        return self.clusters

In [ ]:
proc = MarkovDisseminationProcess(random_points, iterations=200, gamma=-0.02)
proc.run()

Starting a Markov Dissemination Process


100%|██████████| 200/200 [00:47<00:00,  4.20it/s]

Ended Markov Dissemination Process with 18 clusters


In [ ]:
cluster_inverse_dict = proc.generate_cluster_inverse_dict()

data = pd.DataFrame(
    {
        'x': random_points[:, 0],
        'y': random_points[:, 1],
        'cluster': cluster_inverse_dict
    }
)

cluster_points_chart = alt.Chart(data).mark_circle(opacity=0.2).encode(
    x='x:Q',
    y='y:Q',
    color='cluster:N'
)

centers = data.groupby(['cluster'], as_index=False).mean()
# print(centers)
cluster_centers_chart = alt.Chart(centers).mark_point(size=100, opacity=1).encode(
    x='x:Q',
    y='y:Q',
    color='cluster:N',
    shape=alt.value('triangle')
)

cluster_chart = cluster_points_chart+cluster_centers_chart
display(cluster_chart)

alt.LayerChart(...)

In [ ]:
def generate_plot_from_process(proc: MarkovDisseminationProcess) -> alt.Chart:
    cluster_inverse_dict = proc.generate_cluster_inverse_dict()

    data = pd.DataFrame(
        {
            'x': random_points[:, 0],
            'y': random_points[:, 1],
            'cluster': cluster_inverse_dict
        }
    )

    cluster_points_chart = alt.Chart(data).mark_circle(opacity=0.2).encode(
        x='x:Q',
        y='y:Q',
        color='cluster:N'
    )

    centers = data.groupby(['cluster'], as_index=False).mean()
    
    cluster_centers_chart = alt.Chart(centers).mark_point(size=100, opacity=1).encode(
        x='x:Q',
        y='y:Q',
        color='cluster:N',
        shape=alt.value('triangle')
    )

    cluster_chart = cluster_points_chart+cluster_centers_chart
    return cluster_chart

In [ ]:
proc_mean = MarkovDisseminationProcess(random_points, iterations=200, distance_option='mean')
proc_mean.run()
proc_random = MarkovDisseminationProcess(random_points, iterations=200, distance_option='random')
proc_random.run()
proc_min = MarkovDisseminationProcess(random_points, iterations=200, distance_option='min')
proc_min.run()
proc_any = MarkovDisseminationProcess(random_points, iterations=200, distance_option='any')
proc_any.run()

Starting a Markov Dissemination Process


100%|██████████| 200/200 [00:47<00:00,  4.18it/s]


Ended Markov Dissemination Process with 10 clusters
Starting a Markov Dissemination Process


100%|██████████| 200/200 [00:01<00:00, 100.89it/s]


Ended Markov Dissemination Process with 13 clusters
Starting a Markov Dissemination Process


100%|██████████| 200/200 [00:41<00:00,  4.83it/s]


Ended Markov Dissemination Process with 14 clusters
Starting a Markov Dissemination Process


100%|██████████| 200/200 [00:00<00:00, 383.10it/s]

Ended Markov Dissemination Process with 12 clusters


In [ ]:
plot_mean = generate_plot_from_process(proc_mean)
plot_random = generate_plot_from_process(proc_random)
plot_min = generate_plot_from_process(proc_min)
plot_any = generate_plot_from_process(proc_any)
plot = (plot_mean.properties(title='Option=mean') | plot_random.properties(title='Option=random') | plot_min.properties(title='Option=min') | plot_any.properties(title='Option=any')).resolve_scale(color='independent')
display(plot)

alt.HConcatChart(...)

In [ ]:
display(plot_mean)

alt.LayerChart(...)